**Pre-processing**

In [76]:
import pandas as pd
import numpy as np

from azureml.core import Workspace, Dataset, Datastore

subscription_id = 'ed832b48-cccf-4631-b142-e5630bd79da1'
resource_group = 'Resource_Group_1'
workspace_name = 'Workspace1'

workspace = Workspace(subscription_id, resource_group, workspace_name)
datastore = Datastore.get(workspace, 'workspaceblobstore')

dataset = Dataset.get_by_name(workspace, name='account_df')
# dataset2 = Dataset.get_by_name(workspace, name='payment')

account_df = dataset.to_pandas_dataframe()
# payment_df = dataset2.to_pandas_dataframe()


# file1_path = '/content/drive/MyDrive/consumer_debt/account.csv'
# file2_path = '/content/drive/MyDrive/consumer_debt/payments.csv'


# account_df = pd.read_csv(file1_path)
# payment_df = pd.read_csv(file2_path)

In [77]:
# Convert to datetime
account_df['chargeoff_date_bom'] = pd.to_datetime(account_df['chargeoff_date_bom'], dayfirst=True)
account_df['judgment_date'] = pd.to_datetime(account_df['judgment_date'], dayfirst=True)
account_df['min_pay_date'] = pd.to_datetime(account_df['min_pay_date'], dayfirst=True)
account_df['max_pay_date'] = pd.to_datetime(account_df['max_pay_date'], dayfirst=True)
account_df['consumer_dob'] = pd.to_datetime(account_df['consumer_dob'], dayfirst=True)

# payment_df['pay_date'] = pd.to_datetime(payment_df['pay_date'], dayfirst=True)

**Adjustment in account dataset**

In [79]:
# params
# current_date = pd.Timestamp('2024-02-21')  # dataset extraction cutoff
cutoff_date = pd.Timestamp('2019-02-21')  # dataset ready cutoff
# cutoff_days = 1826 # 5 years
# legal_age_min = 16 # add lower limit
# legal_age_max = 65 # add upper limit
# placeholder_date = pd.Timestamp('1970-01-01')

# failure_days = 1825  # if account is still open, but last chargement/ judgement/ payment is beyond this # of days



In [87]:
# Payment amount total to charegoff amount ratio
account_df['avg_pay_to_chargeoff_ratio'] = account_df['avg_pay_amount'] / account_df['chargeoffamt']
# division by zero or nan
account_df['avg_pay_to_chargeoff_ratio'] = account_df['avg_pay_to_chargeoff_ratio'].replace([np.inf, -np.inf], np.nan)

# Payment amount total to judgment amount ratio
account_df['avg_pay_to_judgment_ratio'] = account_df['avg_pay_amount'] / account_df['judgment_amount']
# division by zero or nan
account_df['avg_pay_to_judgment_ratio'] = account_df['avg_pay_to_judgment_ratio'].replace([np.inf, -np.inf], np.nan)

# print((account_df['diff_first_repay_chargeoff'] == 0).sum())

**[obj1] “What are the demographic and
macroeconomic features that are significant in
predicting whether or not a consumer begins
repayment of a debt within 5 years of an initial
chargeoff date?”**

In [88]:
# Target variable for [obj1]: Boolean variable on whether an account receives an initial repayment of a debt
# within 5 years of an initial chargeoff date
account_df['chargeoff_cutoff'] = account_df['chargeoff_date_bom'].apply(lambda x: 'Before' if x <= cutoff_date else 'After' if x >= cutoff_date else 'N/A')


**[subobj1] “Given that a consumer has begun
repayment within 5 years, what is the
probability of that debt being satisfactorily
repaid (paid-in-full or settled-in-full) within 5
years of that initial payment?"**

In [91]:
# Target variable for [subobj1]: Boolean variable on whether an account is being satisfactorily repaid (paid-in-full or settled-in-full)
# within 5 years of its initial repayment date
# account_df['first_repayment_date'] = pd.to_datetime(account_df['first_repayment_date'])
account_df['1stpay_cutoff'] = account_df['min_pay_date'].apply(lambda x: 'Before' if x <= cutoff_date else 'After' )
# NA if there is no first repayment date
account_df.loc[account_df['min_pay_date'].isnull(), '1stpay_cutoff'] = 'N/A'

# NA if first repayment date is smaller than last repayment date
# account_df.loc[account_df['first_repayment_date'] > account_df['last_repayment_date'], '1stpay_cutoff'] = 'N/A'


**Remove unwanted columns**

In [94]:
print(account_df.columns.tolist())


obj1_columns = ['region', 'working_age', 'purchasegroup', \
'chargeoffamt', 'chargeoffamt_present', 'judgment_active', 'acct_post_judgment', 'judgment_amount', 'judgment_amount_present', 'days_between_chargeoff_and_judgment', \
'agency_ever', 'legal_ever', 'wage_garnishable_state', 'bank_account_found', 'FT_job_found', 'PT_job_found', 'verification_completed', 'contains_judgment_image', \
'min_payment_within_5_years_of_chargeoff']

# any payment related columns are removed. this is because any payment related info is the same as telling the model the answer, i.e. they exist if a payment has been made

subobj1_columns = ['region', 'working_age', 'purchasegroup', \
'chargeoffamt', 'chargeoffamt_present', 'judgment_active', 'acct_post_judgment', 'judgment_amount', 'judgment_amount_present', 'days_between_chargeoff_and_judgment', \
'agency_ever', 'legal_ever', 'wage_garnishable_state', 'bank_account_found', 'FT_job_found', 'PT_job_found', 'verification_completed', 'contains_judgment_image', \
'days_between_chargeoff_and_min_pay_date', 'days_between_judgment_and_min_pay_date', \
'avg_days_between_payments', 'avg_monthly_payment(duration>=365days)', \
'avg_pay_to_chargeoff_ratio', 'avg_pay_to_judgment_ratio', \
'closed_w_max_pmt_within_5_yrs_of_min_pmt']

# Anything clearly closure is removed


# remove first repayment date smaller than chargeoff date (diff_first_repay_chargeoff < 0), apply for both models
# account_df = account_df[(account_df['diff_first_repay_chargeoff'] >= 0) | (account_df['first_repayment_date'] == placeholder_date)]

df_1stpay_5yr_chargeoff = account_df.copy()

df_1stpay_5yr_chargeoff_train = df_1stpay_5yr_chargeoff[df_1stpay_5yr_chargeoff['chargeoff_cutoff'] == 'Before'][obj1_columns]
# df_1stpay_5yr_chargeoff_train_prob = df_1stpay_5yr_chargeoff_train.drop('min_payment_within_5_years_of_chargeoff', axis=1)
df_1stpay_5yr_chargeoff_predict_temp = df_1stpay_5yr_chargeoff[df_1stpay_5yr_chargeoff['chargeoff_cutoff'] == 'After'][obj1_columns]
df_1stpay_5yr_chargeoff_predict = df_1stpay_5yr_chargeoff_predict_temp.drop('min_payment_within_5_years_of_chargeoff', axis=1)


# Remove rows where 1stpay_cutoff is 'N/A'
df_close_5yr_1stpay = account_df[account_df['1stpay_cutoff'] != 'N/A']
df_close_5yr_1stpay['closed_w_max_pmt_within_5_yrs_of_min_pmt'] = df_close_5yr_1stpay['closed_w_max_pmt_within_5_yrs_of_min_pmt'].astype('Int64')

df_close_5yr_1stpay_train = df_close_5yr_1stpay[df_close_5yr_1stpay['1stpay_cutoff'] == 'Before'][subobj1_columns]
# df_close_5yr_1stpay_train_prob = df_close_5yr_1stpay_train.drop('t_close_5yr_1stpay', axis=1)
df_close_5yr_1stpay_predict_temp = df_close_5yr_1stpay[df_close_5yr_1stpay['1stpay_cutoff'] == 'After'][subobj1_columns]
df_close_5yr_1stpay_predict = df_close_5yr_1stpay_predict_temp.drop('closed_w_max_pmt_within_5_yrs_of_min_pmt', axis=1)


# full:
# ['data_id', 'region', 'data_state', 'account_status', 'purchasegroup', 'consumer_dob', 'chargeoff_age', 'working_age', 
# 'agency_ever', 'legal_ever', 'wage_garnishable_state', 'bank_account_found', 'FT_job_found', 'PT_job_found', 'verification_completed', 'contains_judgment_image', 
# 'data_bal_total', 'chargeoff_date', 'chargeoff_date_bom', 'chargeoffamt', 'chargeoffamt_present', 
# 'judgment_status', 'judgment_active', 'acct_post_judgment', 'judgment_date', 'days_between_chargeoff_and_judgment', 'judgment_amount', 'judgment_amount_present', 
# 'chargeoff_precovid(2020-03-01)', 'judgment_precovid(2020-03-01)', 
# 'total_transactions', 'num_payments', 'num_reversals', 'net_num_payments', 'sum_pay_amount', 'avg_pay_amount', 
# 'min_pay_date', 'max_pay_date', 'payment_duration_days', 'avg_days_between_payments', 'avg_monthly_payment(duration>=365days)', 
# 'days_between_chargeoff_and_min_pay_date', 'days_between_chargeoff_and_max_pay_date', 'days_between_judgment_and_min_pay_date', 'days_between_judgment_and_max_pay_date', 
# 'min_payment_within_5_years_of_chargeoff', 'closed_w_max_pmt_within_5_yrs_of_min_pmt', 'max_pmt_within_5_yrs_of_min_pmt_within_5_yrs_of_chargeoff', 
# 'avg_pay_to_chargeoff_ratio', 'avg_pay_to_judgment_ratio', 'chargeoff_cutoff', '1stpay_cutoff']
# '''

['data_id', 'region', 'data_state', 'account_status', 'purchasegroup', 'consumer_dob', 'chargeoff_age', 'working_age', 'agency_ever', 'legal_ever', 'wage_garnishable_state', 'bank_account_found', 'FT_job_found', 'PT_job_found', 'verification_completed', 'contains_judgment_image', 'data_bal_total', 'chargeoff_date', 'chargeoff_date_bom', 'chargeoffamt', 'chargeoffamt_present', 'judgment_status', 'judgment_active', 'acct_post_judgment', 'judgment_date', 'days_between_chargeoff_and_judgment', 'judgment_amount', 'judgment_amount_present', 'chargeoff_precovid(2020-03-01)', 'judgment_precovid(2020-03-01)', 'total_transactions', 'num_payments', 'num_reversals', 'net_num_payments', 'sum_pay_amount', 'avg_pay_amount', 'min_pay_date', 'max_pay_date', 'payment_duration_days', 'avg_days_between_payments', 'avg_monthly_payment(duration>=365days)', 'days_between_chargeoff_and_min_pay_date', 'days_between_chargeoff_and_max_pay_date', 'days_between_judgment_and_min_pay_date', 'days_between_judgment_an

/tmp/ipykernel_4165/2549476332.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_close_5yr_1stpay['closed_w_max_pmt_within_5_yrs_of_min_pmt'] = df_close_5yr_1stpay['closed_w_max_pmt_within_5_yrs_of_min_pmt'].astype('Int64')


**Saving output**

In [95]:
from contextlib import suppress

def boolean_conversion(df):
    with suppress(KeyError):
        df['working_age'] = df['working_age'].astype('Int64')
        df['agency_ever'] = df['agency_ever'].astype('Int64')
        df['legal_ever'] = df['legal_ever'].astype('Int64')
        df['wage_garnishable_state'] = df['wage_garnishable_state'].astype('Int64')
        df['bank_account_found'] = df['bank_account_found'].astype('Int64')
        df['FT_job_found'] = df['FT_job_found'].astype('Int64')
        df['PT_job_found'] = df['PT_job_found'].astype('Int64')
        df['verification_completed'] = df['verification_completed'].astype('Int64')
        df['contains_judgment_image'] = df['contains_judgment_image'].astype('Int64')
        df['chargeoffamt_present'] = df['chargeoffamt_present'].astype('Int64')
        df['judgment_active'] = df['judgment_active'].astype('Int64')
        df['acct_post_judgment'] = df['acct_post_judgment'].astype('Int64')
        df['judgment_amount_present'] = df['judgment_amount_present'].astype('Int64')
        df['chargeoff_precovid(2020-03-01)'] = df['chargeoff_precovid(2020-03-01)'].astype('Int64')
        df['judgment_precovid(2020-03-01)'] = df['judgment_precovid(2020-03-01)'].astype('Int64')
        df['min_payment_within_5_years_of_chargeoff'] = df['min_payment_within_5_years_of_chargeoff'].astype('Int64')
        df['closed_w_max_pmt_within_5_yrs_of_min_pmt'] = df['closed_w_max_pmt_within_5_yrs_of_min_pmt'].astype('Int64')
        df['max_pmt_within_5_yrs_of_min_pmt_within_5_yrs_of_chargeoff'] = df['max_pmt_within_5_yrs_of_min_pmt_within_5_yrs_of_chargeoff'].astype('Int64')

        

boolean_conversion(account_df)

boolean_conversion(df_1stpay_5yr_chargeoff)
boolean_conversion(df_1stpay_5yr_chargeoff_train)
boolean_conversion(df_1stpay_5yr_chargeoff_predict)
# boolean_conversion(df_1stpay_5yr_chargeoff_train_prob)

boolean_conversion(df_close_5yr_1stpay)
boolean_conversion(df_close_5yr_1stpay_train)
boolean_conversion(df_close_5yr_1stpay_predict)
# boolean_conversion(df_close_5yr_1stpay_train_prob)

/tmp/ipykernel_4165/3952862573.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['working_age'] = df['working_age'].astype('Int64')
/tmp/ipykernel_4165/3952862573.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['agency_ever'] = df['agency_ever'].astype('Int64')
/tmp/ipykernel_4165/3952862573.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

In [96]:
print(account_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146920 entries, 0 to 146919
Data columns (total 52 columns):
 #   Column                                                     Non-Null Count   Dtype         
---  ------                                                     --------------   -----         
 0   data_id                                                    146920 non-null  int64         
 1   region                                                     146920 non-null  object        
 2   data_state                                                 146920 non-null  object        
 3   account_status                                             146920 non-null  object        
 4   purchasegroup                                              146920 non-null  object        
 5   consumer_dob                                               146920 non-null  datetime64[ns]
 6   chargeoff_age                                              146920 non-null  int64         
 7   working_age         

In [97]:
def count_unique_values(df, column_name, var_name):
    value_counts = df[column_name].value_counts()
    count_table = pd.DataFrame({column_name: value_counts.index, 'Count': value_counts.values})
    # Add a row with the total count
    total_count = df.shape[0]
    count_table.loc[len(count_table)] = ['Total', total_count]
    print(var_name)
    print(count_table)
    print()
    return total_count

print('============ obj1 =============')
total_chargeoff_cutoff = count_unique_values(account_df, 'chargeoff_cutoff', 'account_df')
removed_diff_first_repay_chargeoff = account_df['data_id'].count() - df_1stpay_5yr_chargeoff['data_id'].count()
print('Removed:', removed_diff_first_repay_chargeoff, '\n')

total_obj1_train = count_unique_values(df_1stpay_5yr_chargeoff_train, 'min_payment_within_5_years_of_chargeoff', 'df_1stpay_5yr_chargeoff_train')
total_obj1_predict = count_unique_values(df_1stpay_5yr_chargeoff_predict_temp, 'min_payment_within_5_years_of_chargeoff', 'df_1stpay_5yr_chargeoff_predict')


print('obj1:', total_chargeoff_cutoff, '-', removed_diff_first_repay_chargeoff, '=', total_chargeoff_cutoff - removed_diff_first_repay_chargeoff)
print('obj1:', total_obj1_train, '+', total_obj1_predict, '=', total_obj1_train + total_obj1_predict, '\n')

print('============ subobj1 =============')
total_1stpay_cutoff = count_unique_values(account_df, '1stpay_cutoff', 'account_df')
removed_1stpay_cutoff = account_df['data_id'].count() - df_close_5yr_1stpay['data_id'].count()
print('Removed 1stpay_cutoff = NA:', removed_1stpay_cutoff, '\n')

total_subobj1_train = count_unique_values(df_close_5yr_1stpay_train, 'closed_w_max_pmt_within_5_yrs_of_min_pmt', 'df_close_5yr_1stpay_train')
total_subobj1_predict = count_unique_values(df_close_5yr_1stpay_predict_temp, 'closed_w_max_pmt_within_5_yrs_of_min_pmt', 'df_close_5yr_1stpay_predict')

print('subobj1:', total_1stpay_cutoff, '-', removed_1stpay_cutoff, '=', total_1stpay_cutoff - removed_1stpay_cutoff)
print('subobj1:', total_subobj1_train, '+', total_subobj1_predict, '=', total_subobj1_train + total_subobj1_predict, '\n')

============ obj1 =============
account_df
  chargeoff_cutoff   Count
0           Before  143491
1            After    3429
2            Total  146920

Removed: 0 

df_1stpay_5yr_chargeoff_train
  min_payment_within_5_years_of_chargeoff   Count
0                                       0  119898
1                                       1   23593
2                                   Total  143491

df_1stpay_5yr_chargeoff_predict
  min_payment_within_5_years_of_chargeoff  Count
0                                       0   2437
1                                       1    992
2                                   Total   3429

obj1: 146920 - 0 = 146920
obj1: 143491 + 3429 = 146920 

============ subobj1 =============
account_df
  1stpay_cutoff   Count
0           N/A   81756
1        Before   43461
2         After   21703
3         Total  146920

Removed 1stpay_cutoff = NA: 81756 

df_close_5yr_1stpay_train
  closed_w_max_pmt_within_5_yrs_of_min_pmt  Count
0                                      

In [99]:
# def output_df(df, dataset_name, csv_file_path, target_path='datasets/', datastore=datastore):
def output_df(df, dataset_name, csv_file_path):
    # Register  df as a dataset
    # dataset = Dataset.Tabular.register_pandas_dataframe(df, datastore, dataset_name, show_progress=True)
    # Save the df to a csv file
    df.to_csv(csv_file_path, index=False)
    # df.to_csv('/content/drive/MyDrive/consumer_debt/' + csv_file_path, index=False)

    # Upload the local csv file to the path in datastore
    datastore.upload_files(files=[csv_file_path], target_path='datasets/', overwrite=True, show_progress=True)
    
    dataset = Dataset.Tabular.from_delimited_files(path=(datastore, 'datasets/' + csv_file_path))

# output_df(more_account_df, dataset_name='preprocessed_account_more',  csv_file_path='preprocessed_account_file_more.csv')
# output_df(less_account_df, dataset_name='preprocessed_account_less',  csv_file_path='preprocessed_account_file_less.csv')
# output_df(more_predict_df, dataset_name='preprocessed_account_for_predict_more',  csv_file_path='preprocessed_account_predict_file_more.csv')
# output_df(less_predict_df, dataset_name='preprocessed_account_for_predict_less',  csv_file_path='preprocessed_account_predict_file_less.csv')

output_df(account_df, dataset_name='preprocessed_account_df',  csv_file_path='preprocessed_account_df.csv')

output_df(df_1stpay_5yr_chargeoff, dataset_name='df_1stpay_5yr_chargeoff',  csv_file_path='df_1stpay_5yr_chargeoff.csv')
output_df(df_close_5yr_1stpay, dataset_name='df_close_5yr_1stpay',  csv_file_path='df_close_5yr_1stpay.csv')

output_df(df_1stpay_5yr_chargeoff_train, dataset_name='df_1stpay_5yr_chargeoff_train',  csv_file_path='df_1stpay_5yr_chargeoff_train.csv')
output_df(df_1stpay_5yr_chargeoff_predict, dataset_name='df_1stpay_5yr_chargeoff_predict',  csv_file_path='df_1stpay_5yr_chargeoff_predict.csv')
output_df(df_close_5yr_1stpay_train, dataset_name='df_close_5yr_1stpay_train',  csv_file_path='df_close_5yr_1stpay_train.csv')
output_df(df_close_5yr_1stpay_predict, dataset_name='df_close_5yr_1stpay_predict',  csv_file_path='df_close_5yr_1stpay_predict.csv')

# output_df(df_1stpay_5yr_chargeoff_train_prob, dataset_name='df_1stpay_5yr_chargeoff_train_prob',  csv_file_path='df_1stpay_5yr_chargeoff_train_prob.csv')
# output_df(df_close_5yr_1stpay_train_prob, dataset_name='df_close_5yr_1stpay_train_prob',  csv_file_path='df_close_5yr_1stpay_train_prob.csv')

"datastore.upload_files" is deprecated after version 1.0.69. Please use "FileDatasetFactory.upload_directory" instead. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 1 files
Uploading preprocessed_account_df.csv
Uploaded preprocessed_account_df.csv, 1 files out of an estimated total of 1
Uploaded 1 files
Uploading an estimated of 1 files
Uploading df_1stpay_5yr_chargeoff.csv
Uploaded df_1stpay_5yr_chargeoff.csv, 1 files out of an estimated total of 1
Uploaded 1 files
Uploading an estimated of 1 files
Uploading df_close_5yr_1stpay.csv
Uploaded df_close_5yr_1stpay.csv, 1 files out of an estimated total of 1
Uploaded 1 files
Uploading an estimated of 1 files
Uploading df_1stpay_5yr_chargeoff_train.csv
Uploaded df_1stpay_5yr_chargeoff_train.csv, 1 files out of an estimated total of 1
Uploaded 1 files
Uploading an estimated of 1 files
Uploading df_1stpay_5yr_chargeoff_predict.csv
Uploaded df_1stpay_5yr_chargeoff_predict.csv, 1 files out of an estimated total of 1
Uploaded 1 files
Uploading an estimated of 1 files
Uploading df_close_5yr_1stpay_train.csv
Uploaded df_close_5yr_1stpay_train.csv, 1 files out of an estimated total of